In [ ]:
import os

import sys
sys.path.append("../")

import dgl
import pandas as pd
import torch
import networkx as nx

from graph_conformal.constants import CLASSIFICATION_DATASETS, OGBN_DATASETS, LABEL_FIELD, FEATURE_FIELD, OGBN_ARXIV
from graph_conformal.data_module import init_graph_dataset

In [ ]:
DATASET_ROOT = "../datasets"

columns = ["Nodes", "Edges", "Classes", "Features", "\hat{H}", "H_{rand}"]
data = []
for dataset in [OGBN_ARXIV]:
    print(f"Dataset: {dataset}")
    temp = []
    if dataset in OGBN_DATASETS:
        g, labels = init_graph_dataset(dataset, DATASET_ROOT)[0]
        g.ndata[LABEL_FIELD] = labels.reshape(-1)
    else:
        g = init_graph_dataset(dataset, DATASET_ROOT)[0]

    temp.append(g.num_nodes())
    temp.append(g.num_edges())

    labels = g.ndata[LABEL_FIELD]
    unique_labels, label_counts = labels.unique(return_counts=True)

    temp.append(unique_labels.shape[0])
    temp.append(g.ndata[FEATURE_FIELD].shape[-1])

    nx_g = dgl.to_networkx(g)

    p_k = label_counts / label_counts.sum()
    H = sum(
        (labels[list(nx_g[v])] == labels[v]).sum().item()
        / len(nx_g[v])
        if len(nx_g[v]) else 0
        for v in range(g.num_nodes())
    ) / (g.num_nodes())    
    temp.append(H)
    H_rand = torch.square(p_k).sum().item()

    temp.append(H_rand)
    data.append(temp)

df = pd.DataFrame(data, columns=columns)
df

In [ ]:
df.to_latex()